In [1]:
import numpy as np
import matplotlib.pyplot as plt
from errorPropPendulum import *

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [2]:
blind = True

if blind:
    blinding = np.random.uniform()
else:
    blinding = 0

experiment_by = ["anders", "christine", "Jakob", "makito"]

In [3]:
#Pendulforsøg 1
#Målebåndsmålinger
L_baand_bund = np.array([191.52, 191.7, 192.35, 191.9]) #i cm - rækkefølge: Anders, Makito, Jakob og Christine
L_baand_top = np.array([181.60, 181.60, 182.28, 181.6]) #i cm - rækkefølge: Anders, Makito, Jakob og Christine
L_baand_sigma = 0.1 #i cm

#Lasermålinger
L_laser_bund = np.array([2.004, 2.017, 2.026]) #i m - rækkefølge: Anders, Makito, Jakob
L_laser_top = np.array([1.925, 1.918, 1.922]) #i m - rækkefølge: Anders, Makito, Jakob
L_laser_sigma = 0.001 #i m

Laser_ejenlængde = 10.3 #i cm - medregnet i længderne ovenover

#Længden mellem loddet/pendulet og gulvet blev fastholdt i løbet af forsøget til 21.3 cm, ca. 
#Approkismativt har forsøgsopstillingen ikke ændret sig undervejs

# Afstand til masse midtpunkt antages til at være midt i loddet: pos_mid = (top + bund)/2
# emid = 1/2 sqrt(etop**2 + ebund**2)

# samler og omregner til samme enheder
L_bund = np.concatenate((L_baand_bund/100, L_laser_bund - Laser_ejenlængde/100))
L_top = np.concatenate((L_baand_top/100, L_laser_top - Laser_ejenlængde/100))

# Error antag gaussian
eL_bund = np.std(L_bund)/np.sqrt(len(L_bund))
eL_top = np.std(L_top)/np.sqrt(len(L_top))

L = (np.mean(L_top) + np.mean(L_bund)) / 2 + blinding
eL = 1/2 * np.sqrt(eL_bund**2 + eL_top**2)
print("Pendul forsøg med kort pendul")
print(f"L = {L:.3f} +- {eL:.3f}")

periods = np.array([])
for person in experiment_by:
    data = np.genfromtxt(f"timedat/{person}.dat")
    times = data[:,1]
    Ts = (np.roll(times, -1) - times)[:-1] # T = [t2 - t1, t3 - t2, ..., t1 - tn]  discard the last element

    if person == "anders": # Anders measured everty 5th period
        Ts = Ts/5

    periods = np.append(periods, Ts)

T = np.mean(periods) + blinding
eT = np.std(periods)/np.sqrt(len(periods))
print(f"T = {T:.3f} +- {eT:.3f}")

print(f"Caculated value for g = {eval_g(T, L)} +- {eval_eg(L, T, eL, eT)}")

Pendul forsøg med kort pendul
L = 2.114 +- 0.001
T = 2.996 +- 0.010
Caculated value for g = 9.296596624186739 +- 0.06338939410010729


In [4]:
#Pendulforsøg 2 - langt pendul
#Maalebaandsmaalinger
LP_baand_bund = np.array([18.327, 18.319, 18.330]) #i m - rækkefølge: Anders, Makito, Jakob
LP_baand_top = np.array([18.314, 18.312, 18.313])  #i m - rækkefølge: Anders, Makito, Jakob
L_baand_sigma = 0.1 #i cm. Kommentar: Det er nok lidt for lavt ift. præcisionen af vores måling


#Lasermaalinger
LP_laser_gulv = np.array([18.731, 18.730, 18.747]) #i m - rækkefølge: Anders, Makito, Jakob
#Kommentar: I ovenstående skal laserens ejenlængde ikke trækkes fra
LP_lod_til_gulv = np.array([39.8, 39.8, 40.3]) #i cm - rækkefølge: Anders, Makito, Jakob
L_baand_sigma = 0.1 #i cm
L_laser_sigma = 0.001 #i m

# Afstand til masse midtpunkt antages til at være midt i loddet: pos_mid = (top + bund)/2
# emid = 1/2 sqrt(etop**2 + ebund**2)

# samler og omregner til samme enheder
L_bund = np.concatenate((LP_baand_bund, LP_laser_gulv - LP_lod_til_gulv/100))
L_top = LP_baand_top

# Error antag gaussian
eL_bund = np.std(L_bund)/np.sqrt(len(L_bund))
eL_top = np.std(L_top)/np.sqrt(len(L_top))

L = (np.mean(L_top) + np.mean(L_bund)) / 2 + blinding
eL = 1/2 * np.sqrt(eL_bund**2 + eL_top**2)
print("Pendul forsøg med lang pendul")
print(f"L = {L:.3f} +- {eL:.3f}")


periods = np.array([])
for person in experiment_by:
    if person == "Jakob": # Jakob did not participate
        continue

    data = np.genfromtxt(f"timedat/long_{person}.dat")
    times = data[:,1]
    Ts = (np.roll(times, -1) - times)[:-1] # T = [t2 - t1, t3 - t2, ..., t1 - tn]  discard the last element

    periods = np.append(periods, Ts)


T = np.mean(periods) + blinding
eT = np.std(periods)/np.sqrt(len(periods))
print(f"T = {T:.3f} +- {eT:.3f}")

print(f"Caculated value for g = {eval_g(T, L)} +- {eval_eg(L, T, eL, eT)}")

Pendul forsøg med lang pendul
L = 18.569 +- 0.002
T = 8.835 +- 0.019
Caculated value for g = 9.392199100217 +- 0.040257070656230166


In [ ]:
# time fitting
print("\n  ------------------------------------  Fitting Stage  --------------------------------------- \n")

minFit = 1
maxFit = 100

# Plot data points again
fig_fit = plt.figure(figsize=(16, 8))
frame1 = fig_fit.add_axes((.1,.3,.8,.6))
plt.errorbar(Nworkers[minPlot:maxPlot], Ncompanies[minPlot:maxPlot], yerr=eNcompanies[minPlot:maxPlot], fmt='.', linewidth=2,
                color="blue", label='Data (2020)')
#ax_fit.set(xlabel="Size of company (in FTE)", ylabel="Number of companies", title="")
plt.xlabel( "Size of company (in FTE)",fontsize=16)
plt.ylabel("Number of companies",fontsize=16)
#plt.yscale('log')
plt.legend(fontsize=16);

# Power Law fit:
# --------------
def ChiSquare_exponential(N, alpha):
    Ncompanies_est = func_Exponential(Nworkers[minFit:maxFit], N, alpha)
    chi2 = np.sum(((Ncompanies[minFit:maxFit] - Ncompanies_est) / eNcompanies[minFit:maxFit]) ** 2)
    return chi2

minuit = Minuit(ChiSquare_exponential, N=50000.0, alpha=2.0)
minuit.errordef = 1.0   # Chi2 fit
minuit.migrad();
if (not minuit.fmin.is_valid):
    print("  WARNING: The ChiSquare fit DID NOT converge!!! ")

for name in minuit.parameters:
    print("Fit value (exp): {0} = {1:.5f} +/- {2:.5f}".format(name, minuit.values[name], minuit.errors[name]))
chi2 = minuit.fmin.fval
Ndof = len(Ncompanies[minFit:maxFit]) - len(minuit.values[:])
Prob = stats.chi2.sf(chi2, Ndof)
print(f"  Model fit:  Prob(Chi2={chi2:6.1f}, Ndof={Ndof:3d}) = {Prob:7.5f}")

N_fit, alpha_fit = minuit.values[:]
eN_fit, ealpha_fit = minuit.errors[:]
plt.plot(Nworkers[minFit:maxFit], func_Exponential_vec(Nworkers[minFit:maxFit], N_fit, alpha_fit), 'red', linewidth=2.0,
            label=f'Fit with Exponential (Chi2 = {chi2:6.1f})')

difference = func_Exponential_vec(Nworkers[minFit:maxFit], N_fit, alpha_fit)/ Ncompanies[minFit:maxFit]
frame2 = fig_fit.add_axes((.1,.1,.8,.2))        
plt.plot(Nworkers[minFit:maxFit],difference,'.', color="red")
plt.grid()